In [26]:
import tennis
import simulation_utils
import numpy as np

## Question

A tennis match is played as a series of POINTS, contained within GAMES, which in turn are contained within SETS.

If a player reaches 4 or more points won in a game, and also has won 2 points more than their opponent, then they win that game.

If a player reaches 6 or more games won in a set, and also has won 2 games more than their opponent, then they win that set. If both players reach 6 games, then a TIE-BREAK is played to determine the winner of the set.

Suppose Djokovic plays a set of tennis against Murray under these rules, and his probablity of winning a point is 0.52. Assuming the probability of winning a point remains constant throughout, what would be the probability of Djokovic winnng the set 4-6?

## Constants

In [46]:
POINT_WIN_PROB = 0.52
TARGET_SET_SCORE = (4,6)

## Game winning probability

### Solving for deuce

If $p$ = point winning prob
and $v_{d}(p)$ is game winning prob from deuce


$ v_{d}(p) = p^2*1 + (1-p)^2*0 + 2(p)(1-p)*v_{d}(p)$

This simplifies to
$v_{d}(p) = \frac{p^2}{1-2p(1-p)}$

In [47]:
outcome_prob_finder = tennis.OutcomeProbability(POINT_WIN_PROB,TARGET_SET_SCORE)
print(f'Prob. of win from deuce = {outcome_prob_finder._deuce_win_prob()*100:.1f}%')

Prob. of win from deuce = 54.0%


### Solving for a single game

If $p$ is chance of winning a single point, and $s_{0}$, $s_{1}$ is current score

then can find can find chance of winning game recursively using

$v_{g}(p,s_{0},s_{1}) = $

if $player_{0}$ has already won $\rightarrow 1 $

if $player_{1}$ has already won $\rightarrow 0$

if score is deuce $\rightarrow v_{d}(p)$ 

else $p*v_{g}(p,s_{0}+1,s_{1}) + (1-p)*v_{g}(p,s_{0},s_{1}+1)$


In [48]:
outcome_prob_finder = tennis.OutcomeProbability(POINT_WIN_PROB,TARGET_SET_SCORE)
print(f'Prob. of winning single game from 0-0 = {outcome_prob_finder.win_game_prob()*100:.1f}%')

Prob. of winning single game from 0-0 = 55.0%


### Solving for chance of reaching score

Can use the same recursive approach as above, using the derived $v_{g}(p,0,0)$ to give chance of winning single game.

Here, base cases are returning 1 if target score is reached, or 0 if game ends, or a score higher than the target for one of players is reached

In [49]:
outcome_prob_finder.outcome_prob()

0.09581013757744174

In [44]:
outcome_prob_finder = tennis.OutcomeProbability(POINT_WIN_PROB,TARGET_SET_SCORE)
print(f'Prob. of reaching score {TARGET_SET_SCORE} = {outcome_prob_finder.outcome_prob()*100:.4f}%')

Prob. of reaching score (1, 6) = 2.0432%


## Extension - Baysian updating of point win prob

In practice, our belief of the point winning probability might change as the game progresses. We can model this using a Beta-prior.

https://stats.stackexchange.com/questions/181383/understanding-the-beta-conjugate-prior-in-bayesian-inference-about-a-frequency

We start with a prior belief of the point win prob, and a weight (how quickly we will update this belief).

In [35]:
STRONG_PRIOR = 10000
WEAK_PRIOR = 40

In [36]:
simulations_strong_prior = []
for _ in range(10000):
    simulator = tennis.Bayesian_Game_Simulation(prior=[int(POINT_WIN_PROB*STRONG_PRIOR),
                                                   int((1.0-POINT_WIN_PROB)*STRONG_PRIOR)],
                         target_score=TARGET_SET_SCORE,
                        point_score=[0,0],
                        game_score=[0,0])
    outcome = simulator.sim_outcome()
    simulations_strong_prior.append(outcome)

In [37]:
np.array(simulations_strong_prior).mean()

0.1505

pretty close to constant probability model, because strong prior not updated much

Simulate with weaker prior

In [38]:
simulations_weak_prior = []
for _ in range(10000):
    simulator = tennis.Bayesian_Game_Simulation(prior=[int(POINT_WIN_PROB*WEAK_PRIOR),
                                                   int((1.0-POINT_WIN_PROB)*WEAK_PRIOR)],
                         target_score=TARGET_SET_SCORE,
                        point_score=[0,0],
                        game_score=[0,0])
    outcome = simulator.sim_outcome()
    simulations_weak_prior.append(outcome)

In [39]:
np.array(simulations_weak_prior).mean()

0.1166